In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install PyPDF4
!pip install eng_to_ipa
import eng_to_ipa as eng_to_ipa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyPDF4: filename=PyPDF4-1.27.0-py3-none-any.whl size=61227 sha256=c2a21a3b1c7737364533332d19dd3f79513ea03e4cd8436b625710538102a911
  Stored in directory: /root/.cache/pip/wheels/83/cc/14/cb307e5c99235c4497c7895cdb60b4f7ba2a738b6a5fc0d423
Successfully built PyPDF4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 30.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eng_to_ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822606 sha256=6aa51fbba2facfa474ad148b1cac9e185246d96d510e71db7d9c5b9a6c8a9f3a
  Stored in directory: /root/.cache/pip/wheels/5b/ab/07/fe6722f710d8ef8bd0ccb4eb689ef96f5552f3fc0c80c1aa9c
Successful

In [ ]:
# importing required modules
import pandas as pd
import numpy as np
import nltk
import re
import PyPDF4
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
!pip install wordninja
import wordninja
from itertools import chain
from collections import Counter
import itertools
from nltk import ngrams
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pages_to_remove_astrology = [26,31,36,41,46,51,56,61,66,71,77,82]

In [ ]:
def convert_pdf_to_txt(pdf_directory, start_page,end_page):
    """Convert large pdf file to txt format for NLP corpus
    pdf_directory = directory of pdf file in local disc C or D in quotation mark
    start_page = desired starting page of pdf to extract (int)"""

    pdf_file = open(pdf_directory, 'rb')
    pdf_reader = PyPDF4.PdfFileReader(pdf_file)
    pages = pdf_reader.numPages
    if end_page is None:
      end_page = pages

    for page in range(start_page, end_page):
      if page not in pages_to_remove_astrology:
        page_obj = pdf_reader.getPage(page)
        text = page_obj.extractText()
        file1=open(r"Doc_NLP","a")
        file1.writelines(text)
        #print(text)
      else:
        pass


In [ ]:
#file2=open(r"Doc_NLP_clean","a")
#clean_sentences_with_tab = ['\t' + line for line in clean_sentences]
#with open('Doc_NLP_clean.txt', 'w') as file2:
#    file2.writelines(clean_sentences_with_tab)

In [ ]:
convert_pdf_to_txt('/content/drive/MyDrive/NLP/THE-ONLY-ASTROLOGY-BOOK-YOULL-EVER-NEED-pdf-free-download.pdf', 13,440)

# Text Cleaning

In [ ]:
with open('/content/Doc_NLP', 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
# re patterns
#remove url
url_pattern_1 =r'(http[s]?://)?[a-zA-Z0-9]+([-.][a-zA-Z0-9]+)*\.[a-zA-Z]{2,3}(/\S*)?'
url_pattern_2 = r'http\S+|www.\S+'

#remove '-\n' patterns
hyphen_nl_pattern = r'(-\n)'

#remove hyphens that are found in between a word
hyphen_pattern = r'(?<=[a-z])-(?=[a-z])'

#remove digits
digits = '\d+'

#remove \n
next_line = r'\n'

#remove any symbols
symbols = r'[^\w\s]'

#remove double spaces
double_space = r'\s{2}'


In [ ]:
def sent_preprocess(text):
  sentences = sent_tokenize(text)
  sentences_clean = []
  for sentence in sentences:
    cleaned_sentence = re.sub(url_pattern_1, '', sentence)
    cleaned_sentence = re.sub(url_pattern_2, '', cleaned_sentence)
    cleaned_sentence = re.sub(hyphen_nl_pattern, '', cleaned_sentence)
    cleaned_sentence = re.sub(hyphen_pattern, ' ', cleaned_sentence)
    cleaned_sentence = re.sub(next_line,' ', cleaned_sentence)
    cleaned_sentence = re.sub(digits,'', cleaned_sentence)
    cleaned_sentence = re.sub(symbols,'', cleaned_sentence)
    cleaned_sentence = re.sub(double_space,' ', cleaned_sentence)
    cleaned_sentence = cleaned_sentence.lower()
    cleaned_sentence = cleaned_sentence.strip()
    sentences_clean.append(cleaned_sentence)
  return sentences_clean

In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from unidecode import unidecode


In [ ]:
with open("wrong_words.pickle", "rb") as file:
    wrong_words = pickle.load(file)

In [ ]:
sentences_clean = sent_preprocess(text)

In [ ]:
unigrams_list = []
for sentence in sentences_clean:
    sentence = unidecode(sentence)
    tokens = nltk.tokenize.word_tokenize(sentence)
    filtered_tokens = [word for word in tokens if word not in wrong_words]
    padded = list(pad_sequence(filtered_tokens, pad_left=True, left_pad_symbol="<s>", pad_right=True, right_pad_symbol="</s>", n=2))
    unigrams_list.append(padded)
unigram_list = list(chain.from_iterable(unigrams_list))

In [ ]:
# Dictionary 1
unigram_list_unique = sorted(list(set(unigram_list)))

In [ ]:
unigram_list

['<s>',
 'introduction',
 'xiii',
 'taurus',
 '</s>',
 '<s>',
 'oh',
 'she',
 'said',
 'can',
 'you',
 'tell',
 'me',
 'where',
 'my',
 'three',
 'planets',
 'are',
 'she',
 'did',
 'not',
 'know',
 'that',
 'everyone',
 'has',
 'ten',
 'planets',
 'in',
 'his',
 'or',
 'her',
 'birth',
 'chart',
 '</s>',
 '<s>',
 'but',
 'how',
 'would',
 'my',
 'friend',
 'know',
 'this',
 '</s>',
 '<s>',
 'most',
 'astrology',
 'books',
 'deal',
 'with',
 'birth',
 'charts',
 '</s>',
 '<s>',
 'unless',
 'you',
 'are',
 'willing',
 'to',
 'hunt',
 'down',
 'esoteric',
 'volumes',
 'hidden',
 'in',
 'the',
 'occult',
 'sec',
 'of',
 'bookstores',
 'or',
 'in',
 'special',
 'libraries',
 'where',
 'would',
 'you',
 'go',
 'to',
 'get',
 'that',
 'information',
 '</s>',
 '<s>',
 'any',
 'astrology',
 'book',
 'will',
 'tell',
 'you',
 'whether',
 'your',
 'sun',
 'sign',
 'is',
 'aries',
 'or',
 'pisces',
 'or',
 'one',
 'of',
 'the',
 'other',
 'signs',
 'of',
 'the',
 'zodiac',
 '</s>',
 '<s>',
 'but'

In [ ]:
unigrams_counter = Counter(unigram_list)

In [ ]:
unigrams_counter

Counter({'<s>': 8272,
         'introduction': 4,
         'xiii': 2,
         'taurus': 330,
         '</s>': 8272,
         'oh': 2,
         'she': 476,
         'said': 58,
         'can': 964,
         'you': 5330,
         'tell': 78,
         'me': 16,
         'where': 162,
         'my': 24,
         'three': 132,
         'planets': 462,
         'are': 4226,
         'did': 42,
         'not': 982,
         'know': 244,
         'that': 2348,
         'everyone': 40,
         'has': 720,
         'ten': 66,
         'in': 7244,
         'his': 424,
         'or': 1522,
         'her': 750,
         'birth': 314,
         'chart': 482,
         'but': 402,
         'how': 326,
         'would': 222,
         'friend': 40,
         'this': 1066,
         'most': 420,
         'astrology': 602,
         'books': 34,
         'deal': 140,
         'with': 1972,
         'charts': 44,
         'unless': 26,
         'willing': 46,
         'to': 8388,
         'hunt': 4,
        

In [ ]:
# Dictionary 2 - Save the Counter object to a file
with open('unigrams_counter.pickle', 'wb') as file:
    pickle.dump(unigrams_counter, file)

In [ ]:
bigrams_list = []
for sentence in sentences_clean:
    sentence = unidecode(sentence)
    tokens = nltk.tokenize.word_tokenize(sentence)
    filtered_tokens = [word for word in tokens if word not in wrong_words]
    padded = list(pad_sequence(filtered_tokens, pad_left=True, left_pad_symbol="<s>", pad_right=True, right_pad_symbol="</s>", n=2))
    sentence_bigrams = list(bigrams(padded))
    bigrams_list.append(sentence_bigrams)

In [ ]:
bigram_list = list(chain.from_iterable(bigrams_list))
bigrams_counter = Counter(bigram_list)

BIGRAM PROB DICTIONARY

In [ ]:
bigram_prob_dict = {}
x = 0
for bigram in dict(bigrams_counter):
    w1 = list(dict(bigrams_counter).keys())[x][0]
    w2 = list(dict(bigrams_counter).keys())[x][1]
    bigram_freq = list(dict(bigrams_counter).values())[x]
    unigram_freq = dict(unigrams_counter)[w1]
    # round to 5 decimal places
    bigram_prob_dict[bigram] = round(bigram_freq/unigram_freq, 5)
    x += 1


In [ ]:
bigram_prob_dict

{('<s>', 'introduction'): 0.00024,
 ('introduction', 'xiii'): 0.5,
 ('xiii', 'taurus'): 1.0,
 ('taurus', '</s>'): 0.02424,
 ('<s>', 'oh'): 0.00024,
 ('oh', 'she'): 1.0,
 ('she', 'said'): 0.0084,
 ('said', 'can'): 0.03448,
 ('can', 'you'): 0.00415,
 ('you', 'tell'): 0.00225,
 ('tell', 'me'): 0.02564,
 ('me', 'where'): 0.125,
 ('where', 'my'): 0.01235,
 ('my', 'three'): 0.08333,
 ('three', 'planets'): 0.09091,
 ('planets', 'are'): 0.07792,
 ('are', 'she'): 0.00047,
 ('she', 'did'): 0.0084,
 ('did', 'not'): 0.57143,
 ('not', 'know'): 0.00815,
 ('know', 'that'): 0.08197,
 ('that', 'everyone'): 0.0017,
 ('everyone', 'has'): 0.15,
 ('has', 'ten'): 0.00278,
 ('ten', 'planets'): 0.15152,
 ('planets', 'in'): 0.29437,
 ('in', 'his'): 0.00607,
 ('his', 'or'): 0.14623,
 ('or', 'her'): 0.04731,
 ('her', 'birth'): 0.008,
 ('birth', 'chart'): 0.52229,
 ('chart', '</s>'): 0.14108,
 ('<s>', 'but'): 0.00677,
 ('but', 'how'): 0.00498,
 ('how', 'would'): 0.00613,
 ('would', 'my'): 0.00901,
 ('my', 'friend

In [ ]:
dict(bigrams_counter).get(('will', 'ﬁnd'))

In [ ]:
#Save the Counter object to a file
with open('bigrams_counter.pickle', 'wb') as file:
    pickle.dump(bigrams_counter, file)

In [ ]:
# Dictionary 3 - Save the dict object to a file
with open('bigram_prob_dict_final.pickle', 'wb') as file:
    pickle.dump(bigram_prob_dict, file)

In [ ]:
unigram_list = generate_unigram_list(sentences_clean)

In [ ]:
len(unigram_list)

10910

In [ ]:
unigram_list = list(set(unigram_list))

In [ ]:
# saving unigrams_list into a file
with open("unigram_list.pickle", "wb") as file:
    pickle.dump(unigram_list, file)

In [ ]:
print(unigram_list)

['introduction', 'xiii', 'taurus', 'oh', 'she', 'said', 'can', 'you', 'tell', 'me', 'where', 'my', 'three', 'planets', 'are', 'she', 'did', 'not', 'know', 'that', 'everyone', 'has', 'ten', 'planets', 'in', 'his', 'or', 'her', 'birth', 'chart', 'but', 'how', 'would', 'my', 'friend', 'know', 'this', 'most', 'astrology', 'books', 'dont', 'deal', 'with', 'birth', 'charts', 'unless', 'you', 'are', 'willing', 'to', 'hunt', 'down', 'esoteric', 'volumes', 'hidden', 'in', 'the', 'occult', 'sec', 'tion', 'of', 'bookstores', 'or', 'in', 'special', 'libraries', 'where', 'would', 'you', 'go', 'to', 'get', 'that', 'information', 'any', 'astrology', 'book', 'will', 'tell', 'you', 'whether', 'your', 'sun', 'sign', 'is', 'aries', 'or', 'pisces', 'or', 'one', 'of', 'the', 'other', 'signs', 'of', 'the', 'zodiac', 'but', 'then', 'what', 'this', 'book', 'tells', 'you', 'all', 'you', 'should', 'know', 'about', 'the', 'whole', 'topic', 'of', 'astrology', 'you', 'will', 'learn', 'about', 'sun', 'signs', 'and'

In [ ]:
WORDS = Counter(unigram_list)

In [ ]:
print(WORDS)

Counter({'the': 7219, 'and': 6579, 'of': 4455, 'a': 4319, 'to': 4273, 'in': 4219, 'you': 3563, 'is': 3054, 'are': 2149, 'your': 1867, 's': 1366, 'for': 1268, 'that': 1264, 'be': 1079, 'with': 1032, 'an': 985, 'have': 976, 'or': 929, 'sign': 902, 'as': 832, 'on': 813, 'it': 801, 'this': 671, 'sun': 653, 'at': 582, 'can': 572, 'will': 558, 'by': 546, 'but': 532, 'not': 493, 'love': 486, 're': 481, 'her': 464, 'they': 445, 'also': 435, 'planet': 426, 'moon': 425, 'people': 414, 'may': 394, 'astrology': 388, 'from': 384, 'i': 380, 'has': 378, 'who': 374, 'was': 371, 'what': 368, 'y': 365, 'one': 360, 'their': 352, 'he': 350, 'about': 347, 'signs': 346, 'life': 346, 'if': 343, 'which': 340, 'other': 328, 'd': 328, 'its': 328, 'leo': 327, 'when': 325, 'she': 323, 'house': 321, 'more': 317, 'into': 313, 'chart': 306, 'gemini': 302, 'scorpio': 301, 'through': 300, 'e': 295, 'libra': 291, 'capricorn': 285, 'virgo': 285, 'ce': 282, 'taurus': 279, 'aquarius': 279, 't': 277, 'u': 271, 'en': 269, '

Unigram with IPA

In [ ]:
!pip install eng-to-ipa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eng-to-ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822606 sha256=3299aeeba5722cbbd2e047327d53978f19dc7187d52f859add829253a3240bc9
  Stored in directory: /root/.cache/pip/wheels/5b/ab/07/fe6722f710d8ef8bd0ccb4eb689ef96f5552f3fc0c80c1aa9c
Successfully built eng-to-ipa


In [ ]:
import eng_to_ipa as eng_to_ipa

In [ ]:
unigrams_ipa = []
for i, unigram in enumerate(unigram_list_unique):
    entry = {"token": unigram_list_unique[i]}
    try:
        ipa = eng_to_ipa.convert(eng_to_ipa.convert(unigram))
        if unigram not in ipa:
            entry["ipa"] = ipa
    except Exception as e:
        pass
    unigrams_ipa.append(entry)

unigrams_ipa

[{'token': 'bonding', 'ipa': 'ˈˈbɑndɪŋ*ɪŋ'},
 {'token': 'brooding', 'ipa': 'ˈˈbrudɪŋ*ɪŋ'},
 {'token': 'together', 'ipa': 'təˈgɛðər*'},
 {'token': 'realistic', 'ipa': 'ˌˌriəˈlɪstɪk*'},
 {'token': 'nan', 'ipa': 'næn*'},
 {'token': 'hindus', 'ipa': 'ˈˈhɪnˌduz*'},
 {'token': 'untouchable', 'ipa': 'əənˈtəʧəbəl*'},
 {'token': 'misplacing', 'ipa': 'mɪˈspleɪsɪŋ*ɪŋ'},
 {'token': 'acts', 'ipa': 'æækts*'},
 {'token': 'nuclear', 'ipa': 'ˈˈnukliər*'},
 {'token': 'forehead', 'ipa': 'ˈˈfɔrhɛd*'},
 {'token': 'walls', 'ipa': 'wɔlz*'},
 {'token': 'insensitive', 'ipa': 'ˌɪˌɪnˈsɛnsɪtɪv*'},
 {'token': 'cancelled', 'ipa': 'ˈˈkænsəld*'},
 {'token': 'creator', 'ipa': 'kriˈeɪtər*'},
 {'token': 'dublin', 'ipa': 'ˈˈdəblɪn*'},
 {'token': 'interpretations', 'ipa': 'ˌɪˌɪnˌtərprɪˈteɪʃənz*'},
 {'token': 'birthplace', 'ipa': 'ˈˈbərθˌpleɪs*'},
 {'token': 'footholds', 'ipa': 'ˈˈfʊˌthoʊldz*'},
 {'token': 'ziggurat', 'ipa': 'ˈˈzɪgərˌæt*'},
 {'token': 'ix'},
 {'token': 'takes', 'ipa': 'teɪks*'},
 {'token': 'fact', 'ipa': '

In [ ]:
 #Dictionary 4 - saving unigrams_ipa into a file
import json

with open('unigrams_ipa.json', 'w') as f:
    json.dump(unigrams_ipa, f)